#### 추천 시스템 방식
- 콘텐츠 기반 필터링
- 협업 필터링
- 하이브리드
#### 관련연구 및 제안 사항
- => 정보의 홍수 속에서 각 개인의 선호도에 따라 적절한 컨텐츠를 추천해 주는 서비스에 매력을 느낄 것이다

- 사용자/아이템 기반의 협업 피터링
    - : 나와 선호도가 유사한 사용자들을 기반으로 내가 접하지 않았던 아이템들에 대한 선호도를 예측하는 기법
    - 장점 : 예상하지 못한 아이템들을 추천 받을 수 있다
    - 단점 : 선호도 측정을 위한 데이터를 축적해야함. 
             새로운 사용자나 컴텐츠에 대해 추천결과에 반영 시킬 수 없는 콜드 스타트 문제

- 컨텐츠 기반의 추천
    * 평소에 자주 접하던 아이템을 분석하여 유사한 아이템들을 추천. 
    * 특정 컨텐츠와 유사한 성질을 가지는 컨텐츠를 검색 -> 전통적인 정보 검색에 근거
    - 장점 : 사용자가 선호하는 아이템들을 선별하여 추천해주기 때문에 일정수준 이상의 사용자 경험(UX) 만족도 보장
    - 단점 : 같은 주제를 가지는 뉴스만 추천 -> 피로도와 지루함을 쉽게 느낌

### 컨텐츠 기반 필터링 실습 - TMDB 5000 Movie Dataset

In [2]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('../data/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


* id
* title : 제목
* genres : 영화 장르
* vote_average : 평균평점
* vote_count : 평점 카운트
* popularity : 인기도
* keywords : 영화의 키워드
* overview : 개요 설명

In [3]:
movies_df = movies[['id', 'title', 'genres','vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

In [4]:
pd.set_option('max_colwidth', 100)
# 리스트 안에 딕셔너리로 들어가 있다
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [5]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


텍스트 문자 1차 가공. 파이썬 딕셔너리 변환 후 리스트로 변환

In [8]:
# 텍스트를 자동으로 파싱해서 파이썬 객체로 만들어 줌
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

# 딕셔너리에 name 키로 가지고 있는 value로 추출 리스트 컴프리핸션으로 
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])

movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


장르 콘텐츠 필터링을 이용한 영화 추천. 장르 문자열을 Count벡터화 후에 코사인 유사도로 각 영화를 비교 

장르 문자열의 Count기반 feature 벡터화

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백 문자로 word단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))

# min-df : 최소빈도수, n-gram이라는 것은 단어의 묶음
# ngram_range = (1,2)라고 한다면 단어의 묶음을 1개부터 2개까지 설정
count_vect = CountVectorizer(min_df = 0, ngram_range = (1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [10]:
movies_df['genres_literal']

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4798                       Action Crime Thriller
4799                              Comedy Romance
4800               Comedy Drama Romance TV Movie
4801                                            
4802                                 Documentary
Name: genres_literal, Length: 4803, dtype: object

장르에 따른 영화별 코사인 유사도 추출

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
genre_sim = cosine_similarity(genre_mat, genre_mat)

print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [12]:
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


In [14]:
a = np.array([1.5, 0.2, 4.2, 2.5])
print(a)
print('인덱스 정렬:', a.argsort())   # 인덱스 정렬
print('역순정렬',a[::-1])
print('인덱스 역순 정렬:', a.argsort()[::-1]) 

[1.5 0.2 4.2 2.5]
인덱스 정렬: [1 0 3 2]
역순정렬 [2.5 4.2 0.2 1.5]
인덱스 역순 정렬: [2 3 0 1]


In [19]:
print(genre_sim[:2])
print('---------------------------------')
print(np.argsort(genre_sim[:2]))
#작은 값부터 인덱스 위치 반환
print('---------------------------------')
print([genre_sim[:1][::-1]])
#처음부터 끝까지 -1칸 간격으로 ( == 역순으로)

[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]
---------------------------------
[[2401 3037 3038 ...  813 3494    0]
 [2401 3067 3069 ...  129    1  262]]
---------------------------------
[array([[1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
        0.        ]])]


특정 영화와 장르별 유사도가 높은 영화를 반환하는 함수 생성

In [17]:
def find_sim_movies(df, sorted_ind, title_name, top_n = 10):
    # 인자로 입력된 'movies_df' df에서 'title' 칼럼이 입력된 'title_name'값인 df 추출
    title_movie = df[df['title'] == title_name]
    
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    # 추출된 top_n개의 indexes를 출력. top_n indexes는 2차원 데이터
    # dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

In [18]:
similar_movies = find_sim_movies(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


평점이 높은 영화 정보 확인

In [20]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending = False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


* 영화 평점(vote)와 관련 평균평점(vote_average) - 불공정
* 평점을 남긴 count(vote_count) 3개 - 다 5점
* vote가 많을수록 평점이 5점이 나올 수 없고, 떨어질 수 있다.
* 참조 https://www.quora.com/How-does-IMDbs-rating-system-work

**평가 횟수에 대한 가중치가 부여된 평점(Weighted Rating) 계산 <br>
         가중 평점(Weighted Rating) = (v/(v+m)) * R + (m/(v+m)) * C**
*  v: 개별 영화에 평점을 투표한 횟수
* m: 평점을 부여하기 위한 최소 투표 횟수(ex.상위 60%, 40%)
* R: 개별 영화에 대한 평균 평점.
* C: 전체 영화에 대한 평균 평점

In [23]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)  # 상위 60%
print('C :',round(C,3), 'm :',round(m,3))

C : 6.092 m : 370.2


In [24]:
percentile = 0.6
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(percentile)

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   #가중 평점

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [25]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending = False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [28]:
def find_sim_movies(df, sorted_ind, title_name, top_n = 10):
    # 인자로 입력된 'movies_df' df에서 'title' 칼럼이 입력된 'title_name'값인 df 추출
    title_movie = df[df['title'] == title_name]
    
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    # 추출된 top_n개의 indexes를 출력. top_n indexes는 2차원 데이터
    # dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    similar_indexes = similar_indexes.reshape(-1)
    
    # 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending = False)[:top_n]

In [29]:
similar_movies = find_sim_movies(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


In [30]:
similar_movies = find_sim_movies(movies_df, genre_sim_sorted_ind, 'Mulan', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
328,Finding Nemo,7.6,7.514020
874,Anastasia,7.4,7.131352
34,Monsters University,7.0,6.913786
430,Lilo & Stitch,7.1,6.878472
137,Kung Fu Panda 2,6.7,6.600001
54,The Good Dinosaur,6.6,6.510741
130,Bolt,6.3,6.263712
2443,Dragon Hunters,6.5,6.197560
399,Open Season,6.1,6.097176
1950,The True Story of Puss 'n Boots,3.8,6.049634
